In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import bioinfokit.analys
from sklearn.metrics import mean_absolute_error as score_MAE
from sklearn.metrics import mean_squared_error as score_MSE 
from sklearn.metrics import median_absolute_error as score_MED
from sklearn.metrics import r2_score as score_R2
import matplotlib.gridspec as gridspec

## Importing Data and variable declaration

In [ ]:
###  Macaca fascicularis batch-corrected CPM
file = './source_data/batch_correction/batch_cor_GeneCPM_mfa.txt'
GeneCPM = pd.read_csv(file, sep = '\t', header = 0)
GeneCPM = GeneCPM.set_index('gene_chr')

###  Macaca mulatta CPM
file = './source_data/count_result/GeneCPM_mml.txt'
GeneCPM_mml = pd.read_csv(file, sep = '\t', header = 0)
GeneCPM_mml = GeneCPM_mml.set_index('gene_chr')

#########################  HUMAN_Mendeley_2020_data
#### Genecount_CPM
Men_gene_read = pd.read_csv('./source_data/Mendeley_2020/raw_filtered.txt', sep = '\t', header = 0)
Men_gene_read = Men_gene_read.rename(columns={'Unnamed: 0':'Name'})

Men_sample = pd.read_csv('./source_data/Mendeley_2020/meta_filtered.txt', sep = '\t', header = 0)

#########################  TRAP_GENE

TRAP = pd.read_excel('./source_data/AGE_predition/TRAP_R/41467_2015_BFncomms9570_MOESM436_ESM.xlsx',header=0,skiprows=2)
TRAP_GENE_DISC = TRAP[TRAP['P'] < 0.05]
TRAP_GENE_META = TRAP[TRAP['META: P'] < 0.05]

#########################  PREDICTOR
predictor = pd.read_excel('./source_data/AGE_predition/TRAP_R/41467_2015_BFncomms9570_MOESM440_ESM.xlsx', sheet_name='Data.5.B',skiprows=3)
predictor = predictor[['GeneID','PREDICTOR-for-NEW-COHORTS']]

In [ ]:
#########################  AGE

### Macaca fascicularis

file = './source_data/reference/mfa_samplelist.txt'
mfa_Con = pd.read_csv(file, sep = '\t', header = None, names=['SAMPLEID'])
mfa_Con['AGE'] = [3,3,7,7,12,12,17,17,4,4,8,8,13,13,18,18,5,5,9,9,14,14,19,19]
mfa_Con = mfa_Con.sort_values(by=['AGE','SAMPLEID'], ascending=[True,True])
#mfa_Con.to_csv('/home/chm/analyzing_file_byR/RNA/SQuIRE_count_ELTC/AGE_predition/TRAP_R/SAMPLEID_AGE.txt',sep='\t',index=False,header=True)

GeneCPM = GeneCPM[mfa_Con['SAMPLEID'].tolist()]

### Macaca mulatta

file = './source_data/reference/mml_condition.txt'
mml_Con = pd.read_csv(file, sep = ' ', header = None, names=['SAMPLEID','AGE','ID','SEX'])
mml_Con = mml_Con[mml_Con['SEX']=='male']
mml_Con = mml_Con[['SAMPLEID','AGE']]
mml_Con = mml_Con.sort_values(by=['AGE','SAMPLEID'], ascending=[True,True])

GeneCPM_mml = GeneCPM_mml[mml_Con['SAMPLEID'].tolist()]

In [ ]:
#########################  AGE

### Homo sapience

GeneRaw_hsa = Men_gene_read.rename(columns={'Unnamed: 0':'Name'})
GeneRaw_hsa = GeneRaw_hsa.set_index('Name')

### 

SEL_healthy = Men_sample[(Men_sample['Gender'] == 'Male') &(Men_sample['Tissue'].str.contains('Blood;PBMC')) & (Men_sample['Condition'] == 'Healthy')]

h_hsa_Con = SEL_healthy[['SRR.ID','Age']]
h_hsa_Con = h_hsa_Con.rename(columns={'SRR.ID':'SAMPLEID','Age':'AGE'})

GeneRaw_hhsa = GeneRaw_hsa[h_hsa_Con['SAMPLEID'].tolist()]

### CPM_normalization

nm = bioinfokit.analys.norm()
nm.cpm(df=GeneRaw_hhsa)
GeneRaw_hhsa = nm.cpm_norm

In [ ]:
hubgene = pd.read_csv('./source_data/hubgene/hubgene_20.txt', sep = '\t', header = 0)
hubgene_all = pd.read_csv('./source_data/hubgene/hubgene_605.txt', sep = '\t', header = 0)
hubgene_cancer = pd.read_csv('./source_data/hubgene/hubgene_cancer.txt', sep = '\t', header = 0)
#MCC_hubgene
#hubgene_13 = hubgene[hubgene['Name'].isin(['MYC','IRF4','CD28','IL7R','CCR7',
#                                           'PRF1','FAS',
#                                           'CD79B','CD79A','PAX5'])]


#hubgene_13.to_csv('/home/chm/analyzing_file_byR/RNA/SQuIRE_count_ELTC/Final_data/hubgene_13.txt',sep='\t',index=False,header=True)

In [ ]:
hubgene_rhesus = pd.read_csv('./source_data/hubgene/biomaRt/hubgene_20_rhesus.txt', sep = '\t', header = 0)
hubgene_all_rhesus = pd.read_csv('./source_data/hubgene/biomaRt/hubgene_605_rhesus.txt', sep = '\t', header = 0)
hubgene_cancer_rhesus =  pd.read_csv('./source_data/hubgene/biomaRt/hubgene_cancer_rhesus.txt', sep = '\t', header = 0)

In [ ]:
hubgene_human = pd.read_csv('./source_data/hubgene/biomaRt/hubgene_20_human.txt', sep = '\t', header = 0)
hubgene_all_human = pd.read_csv('./source_data/hubgene/biomaRt/hubgene_605_human.txt', sep = '\t', header = 0)
hubgene_cancer_human =  pd.read_csv('./source_data/hubgene/biomaRt/hubgene_cancer_human.txt', sep = '\t', header = 0)

In [ ]:
def formula(dt,dt_T):

    DF = pd.DataFrame()

    for i in predictor.index:
        gene = predictor.loc[i]['GeneID']
        value = predictor.loc[i]['PREDICTOR-for-NEW-COHORTS']
    
        if gene in dt.index:
            DF[gene] = dt_T[gene] * value
        else:
            continue
    return DF

In [ ]:
def parsing_formula(dt,dt_filterlist):
    
    
    dt['GENEID'] = dt.index.str.split('_').str[0]
    dt = dt.set_index('GENEID')
    dt = dt.reset_index(drop=False)
    
    dt = dt.groupby(['GENEID'])[dt.columns.tolist()].sum()
    dt = dt.reset_index(drop=False)

    dt = dt[dt['GENEID'].isin(dt_filterlist)]
    dt = dt.set_index('GENEID')
    dt_T = dt.T
    
    dt_T_f = formula(dt,dt_T) 
    
    return dt_T_f

In [ ]:
D_mfa = parsing_formula(GeneCPM,TRAP_GENE_DISC['NEW-Gene-ID'].unique())
M_mfa = parsing_formula(GeneCPM,TRAP_GENE_META['NEW-Gene-ID'].unique())

D_mml = parsing_formula(GeneCPM_mml,TRAP_GENE_DISC['NEW-Gene-ID'].unique())
M_mml = parsing_formula(GeneCPM_mml,TRAP_GENE_META['NEW-Gene-ID'].unique())

ND_hsa = parsing_formula(GeneRaw_hhsa,TRAP_GENE_DISC['NEW-Gene-ID'].unique())
NM_hsa = parsing_formula(GeneRaw_hhsa,TRAP_GENE_META['NEW-Gene-ID'].unique())

In [ ]:
def mfa_mml_hub(df,df_rhesus):

    #its = list(set(D_mfa.columns.tolist()) & set(df['Name'].tolist()))
    its = list(set(predictor['GeneID'].unique()) & set(df['gene'].tolist()))
    #mfa_hub = D_mfa[its]
    mfa_hub = parsing_formula(GeneCPM,its)



    #its = list(set(D_mml.columns.tolist()) & set(df_rhesus['Name'].tolist()))
    its = list(set(predictor['GeneID'].unique()) & set(df['gene'].tolist()))
    #mml_hub = D_mml[its]
    mml_hub = parsing_formula(GeneCPM_mml,its)

    
    return mfa_hub, mml_hub


def hsa_hub(df):

    its = list(set(predictor['GeneID'].unique()) & set(df['gene'].tolist()))
    nha_hub = parsing_formula(GeneRaw_hhsa,its)

    return nha_hub


def prediction(dt,chron_age):

    dt_sum = dt.sum(axis=1)
    
    mean_pred = dt_sum.mean()
    std_pred = dt_sum.std()
 
    mean_chron = chron_age['AGE'].mean()
    std_chron = chron_age['AGE'].std()

    age = []
    for i in dt_sum.index:
        predictor = dt_sum.loc[i]
        pred_age = mean_chron + (predictor-mean_pred) * std_chron / std_pred
        
        if pred_age < 0:
            pred_age = 0
        else:
            pass
        age.append(pred_age)
        
    return age



def merge(a,b,c,con):

    A = prediction(a,con)
    D = prediction(b,con)
    M = prediction(c,con)

    con['ALL_hub_age'] = A
    con['DISC_age'] = D
    con['META_age'] = M   
    
    return con



mfa_hubgene_cancer, mml_hubgene_cancer = mfa_mml_hub(hubgene_cancer,hubgene_cancer_rhesus)
mfa_hubgene, mml_hubgene = mfa_mml_hub(hubgene,hubgene_rhesus)
mfa_hubgene_all, mml_hubgene_all = mfa_mml_hub(hubgene_all,hubgene_all_rhesus)

nha_hubgene_cancer = hsa_hub(hubgene_cancer_human)
nha_hubgene = hsa_hub(hubgene_human)
nha_hubgene_all = hsa_hub(hubgene_all_human)


###
MF = merge(mfa_hubgene,D_mfa,M_mfa,mfa_Con)
MFA = MF.rename(columns={'ALL_hub_age':'hub_age'})
ML = merge(mml_hubgene,D_mml,M_mml,mml_Con) 
MLL = ML.rename(columns={'ALL_hub_age':'hub_age'})

NH = merge(nha_hubgene,ND_hsa,NM_hsa,h_hsa_Con) 
NHA = NH.rename(columns={'ALL_hub_age':'hub_age'})


###
MF_cancer = merge(mfa_hubgene_cancer,D_mfa,M_mfa,mfa_Con)
MFA_cancer = MF_cancer.rename(columns={'ALL_hub_age':'hub_age_cancer'})
ML_cancer = merge(mml_hubgene_cancer,D_mml,M_mml,mml_Con) 
MLL_cancer = ML_cancer.rename(columns={'ALL_hub_age':'hub_age_cancer'})

NH_cancer = merge(nha_hubgene_cancer,ND_hsa,NM_hsa,h_hsa_Con) 
NHA_cancer = NH_cancer.rename(columns={'ALL_hub_age':'hub_age_cancer'})


###
MF_all = merge(mfa_hubgene_all,D_mfa,M_mfa,mfa_Con)
MFA_all = MF_all.rename(columns={'ALL_hub_age':'hub_age_all'})
ML_all = merge(mml_hubgene_all,D_mml,M_mml,mml_Con)
MLL_all = ML_all.rename(columns={'ALL_hub_age':'hub_age_all'})

NH_all = merge(nha_hubgene_all,ND_hsa,NM_hsa,h_hsa_Con) 
NHA_all = NH_all.rename(columns={'ALL_hub_age':'hub_age_all'})


MFA_All = pd.concat([MFA_cancer,MFA[['hub_age']],MFA_all[['hub_age_all']]],axis=1) 
MML_All = pd.concat([MLL_cancer,MLL[['hub_age']],MLL_all[['hub_age_all']]],axis=1)

NHA_All = pd.concat([NHA_cancer,NHA[['hub_age']],NHA_all[['hub_age_all']]],axis=1) 

In [ ]:
def age_graph(dt,min_age,max_age,Y_max,S):
    
    true_age = dt['AGE']
    pred_age_1 = dt['DISC_age']
    pred_age_2 = dt['META_age']
    pred_age_3 = dt['hub_age_cancer']
    pred_age_4 = dt['hub_age']
    pred_age_5 = dt['hub_age_all']

    err_1 = "MAE:{:3.1f}  MED:{:3.1f}  R2:{:3.2f}".format( score_MAE(true_age,pred_age_1), score_MED(true_age,pred_age_1), score_R2(true_age,pred_age_1))
    err_2 = "MAE:{:3.1f}  MED:{:3.1f}  R2:{:3.2f}".format( score_MAE(true_age,pred_age_2), score_MED(true_age,pred_age_2), score_R2(true_age,pred_age_2))
    err_3 = "MAE:{:3.1f}  MED:{:3.1f}  R2:{:3.2f}".format( score_MAE(true_age,pred_age_3), score_MED(true_age,pred_age_3), score_R2(true_age,pred_age_3))
    err_4 = "MAE:{:3.1f}  MED:{:3.1f}  R2:{:3.2f}".format( score_MAE(true_age,pred_age_4), score_MED(true_age,pred_age_4), score_R2(true_age,pred_age_4))    
    err_5 = "MAE:{:3.1f}  MED:{:3.1f}  R2:{:3.2f}".format( score_MAE(true_age,pred_age_5), score_MED(true_age,pred_age_5), score_R2(true_age,pred_age_5))    
        
    
    sns.set_style('white')
    sns.set_palette( sns.xkcd_palette(['nice blue','faded red']))
    
    fig = plt.figure(figsize=(15,3.3))
    #fig = plt.figure(figsize=(13,3.5))
    
    ax1 = fig.add_subplot(1,5,1)
    ax1.plot([3,max_age],[3,max_age],'k:')

    sns.regplot(data=dt, x='AGE', y='DISC_age', ax=ax1, color='orange')
    
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        ax1.text(0.95, 0.03, err_1, horizontalalignment='right',verticalalignment='bottom',transform=ax1.transAxes)
    else:
        ax1.text(1,0, err_1, horizontalalignment='right',verticalalignment='bottom',transform=ax1.transAxes)

    if S == 'Macaca fascicularis':
        ax1.text(12,25,'DISC', ha='center', va='center', size=20,color = 'orange')
    if S == 'Normal Human':
        ax1.text(35,60,'DISC', ha='center', va='center', size=20,color = 'orange')
        
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        ax1.text(-7, 12, S , ha='center', va='center', size=20,  rotation=90)
    else:
        ax1.text(-4.5, 35, S , ha='center', va='center', size=20,  rotation=90)

    plt.axis('equal')
    ax1.set_xlabel('True age (years)')
    ax1.set_ylabel('Predicted age (years)')
    sns.despine()
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        plt.ylim(0,25)       
    else:
        plt.ylim(20,Y_max)       
    plt.xlim(min_age,max_age)
    plt.tight_layout()
    

    ax2 = fig.add_subplot(1,5,2)
    ax2.plot([3,max_age],[3,max_age],'k:')

    sns.regplot(data=dt, x='AGE', y='META_age', ax=ax2, color='deepskyblue')
    
    
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        ax2.text(0.95, 0.03, err_2, horizontalalignment='right',verticalalignment='bottom',transform=ax2.transAxes)
    else:
        ax2.text(1, 0, err_2, horizontalalignment='right',verticalalignment='bottom',transform=ax2.transAxes)

    if S == 'Macaca fascicularis':
        ax2.text(11.9,25,'META', ha='center', va='center', size=20,color = 'deepskyblue')
    if S == 'Normal Human':
        ax2.text(35,56,'META', ha='center', va='center', size=20,color = 'deepskyblue')

    plt.axis('equal')
    ax2.set_xlabel('True age (years)')
    #ax2.set_ylabel('Predicted age (years)')
    plt.ylabel('')
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        plt.ylim(0,25)       
    else:
        plt.ylim(20,Y_max)       
    plt.xlim(min_age,max_age)
    sns.despine()
    plt.tight_layout()
    
    
    ax3 = fig.add_subplot(1,5,3)
    ax3.plot([3,max_age],[3,max_age],'k:')

    sns.regplot(data=dt, x='AGE', y='hub_age_cancer', ax=ax3, color='darksalmon')
    
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        ax3.text(0.95, 0.03, err_3, horizontalalignment='right',verticalalignment='bottom',transform=ax3.transAxes)
    else:
        ax3.text(1,0, err_3, horizontalalignment='right',verticalalignment='bottom',transform=ax3.transAxes)
    
    if S == 'Macaca fascicularis':
        ax3.text(14,25,'HUB_cancer', ha='center', va='center', size=20,color = 'darksalmon')
    if S == 'Normal Human':
        ax3.text(36,56,'HUB_cancer', ha='center', va='center', size=20,color = 'darksalmon')
   
    plt.axis('equal')
    ax3.set_xlabel('True age (years)')
    #ax3.set_ylabel('Predicted age (years)')
    plt.ylabel('')
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        plt.ylim(0,25)       
    else:
        plt.ylim(20,Y_max)       
    plt.xlim(min_age,max_age)
    sns.despine()
    plt.tight_layout()
    
    
    ax4 = fig.add_subplot(1,5,4)
    ax4.plot([3,max_age],[3,max_age],'k:')

    sns.regplot(data=dt, x='AGE', y='hub_age', ax=ax4, color='darksalmon')
    
    
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        ax4.text(0.95, 0.03, err_4, horizontalalignment='right',verticalalignment='bottom',transform=ax4.transAxes)
    else:
        ax4.text(1,0, err_4, horizontalalignment='right',verticalalignment='bottom',transform=ax4.transAxes)
  
    if S == 'Macaca fascicularis':
        ax4.text(12,25,'HUB_20', ha='center', va='center', size=20,color = 'darksalmon')
    if S == 'Normal Human':
        ax4.text(35,56,'HUB_20', ha='center', va='center', size=20,color = 'darksalmon')
   
    plt.axis('equal')
    ax4.set_xlabel('True age (years)')
    #ax4.set_ylabel('Predicted age (years)')
    plt.ylabel('')
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        plt.ylim(0,25)       
    else:
        plt.ylim(20,Y_max)       
    plt.xlim(min_age,max_age)
    sns.despine()
    plt.tight_layout()
    
    
    ax5 = fig.add_subplot(1,5,5)
    ax5.plot([3,max_age],[3,max_age],'k:')

    sns.regplot(data=dt, x='AGE', y='hub_age_all', ax=ax5, color='darksalmon')
    
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        ax5.text(0.95, 0.03, err_5, horizontalalignment='right',verticalalignment='bottom',transform=ax5.transAxes)
    else:
        ax5.text(1, 0, err_5, horizontalalignment='right',verticalalignment='bottom',transform=ax5.transAxes)
    
    if S == 'Macaca fascicularis':
        ax5.text(12,25,'HUB_all', ha='center', va='center', size=20,color = 'darksalmon')
    if S == 'Normal Human':
        ax5.text(35,56,'HUB_all', ha='center', va='center', size=20,color = 'darksalmon') 
   
    plt.axis('equal')
    ax5.set_xlabel('True age (years)')
    #ax5.set_ylabel('Predicted age (years)')
    plt.ylabel('')
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        plt.ylim(0,25)       
    else:
        plt.ylim(20,Y_max)       
    plt.xlim(min_age,max_age)
    sns.despine()
    plt.tight_layout()
    
    #plt.savefig("./figures/"+S+"_transcriptomeAGE.png",dpi=1000)
    plt.show()


In [ ]:
age_graph(MFA_All,8,20,'NA','Macaca fascicularis')
plt.show()

In [ ]:
age_graph(MML_All,3,23,'NA','Macaca mulatta')
plt.show()

In [ ]:
age_graph(NHA_All,20,50,50,'Normal Human')
plt.show()